In [2]:
import tqdm
import fiftyone as fo
import fiftyone.brain as fob
from fiftyone.types import FiftyOneVideoLabelsDataset
import fiftyone.utils.video
import fiftyone.utils.torch as fout
from fiftyone import ViewField as F
# /home/bea/miniconda3/envs/ptg/lib/python3.10/site-packages/fiftyone/brain/internal/core/utils.py
# /home/bea/miniconda3/envs/ptg/lib/python3.10/site-packages/fiftyone/brain/internal/core/visualization.py

In [3]:
RESOURCES_DIR = '/datasets'

eta_dataset = '/datasets/Milly'
export_dataset = '/datasets/Milly/object_labels'
name = 'milly'
max_samples=None

In [4]:
Importer = FiftyOneVideoLabelsDataset().get_dataset_importer_cls()
video_dataset = fo.Dataset.from_importer(Importer(eta_dataset, max_samples=max_samples))

 100% |███████████████████| 10/10 [1.6m elapsed, 0s remaining, 0.1 samples/s]     


In [5]:
frames_dataset = video_dataset.to_frames(sample_frames=True)
frames_dataset.persistent = True

Computing metadata...
 100% |███████████████████| 10/10 [492.1ms elapsed, 0s remaining, 20.3 samples/s] 
Setting 48596 frame filepaths on the input collection that exist on disk but are not recorded on the dataset


In [6]:
sim_idx = fiftyone.brain.compute_similarity(
    frames_dataset, 
    patches_field="detections", 
    brain_key="emb_sim",
    model="clip-vit-base32-torch",
    batch_size=256, 
    backend='lancedb',
)

Computing patch embeddings...
  98% |████████████\| 47732/48596 [3.3m elapsed, 4.0s remaining, 215.4 samples/s]  

In [ ]:
results = fob.compute_visualization(
    frames_dataset, 
    embeddings=sim_idx,
    patches_field="detections", 
    model="clip-vit-base32-torch",
    brain_key="emb_viz",
    batch_size=256, 
)

Generating visualization...


/home/bea/.local/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/bea/.local/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/bea/.local/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopyth

UMAP( verbose=True)
Wed Aug  9 10:28:35 2023 Construct fuzzy simplicial set
Wed Aug  9 10:28:35 2023 Finding Nearest Neighbors
Wed Aug  9 10:28:35 2023 Building RP forest with 24 trees
Wed Aug  9 10:28:40 2023 NN descent for 17 iterations
	 1  /  17
	 2  /  17
	 3  /  17
	Stopping threshold met -- exiting after 3 iterations
Wed Aug  9 10:28:53 2023 Finished Nearest Neighbor Search
Wed Aug  9 10:28:55 2023 Construct embedding


Epochs completed: 100%| ██████████ 200/200 [00:25]


Wed Aug  9 10:29:52 2023 Finished embedding


In [ ]:
# fob.compute_uniqueness(frames_dataset, roi_field='detections', embeddings=sim_idx)

In [ ]:
# counts = frames_dataset.count_values("detections.detections.label")
# classes = sorted(counts, key=counts.get, reverse=True)[:10]
# view = frames_dataset.filter_labels("detections", F("label").is_in(classes))

In [ ]:
patches_dataset = frames_dataset.to_patches('detections')

In [10]:
fob.compute_uniqueness(patches_dataset)

Computing embeddings...
 100% |███████████| 149049/149049 [5.3m elapsed, 0s remaining, 502.6 samples/s]      
Computing uniqueness...
Computing neighbors for 149049 embeddings; this may take awhile...
Uniqueness computation complete


In [11]:
view = patches_dataset

In [12]:
view

Dataset:     2023.08.09.10.23.06
Media type:  image
Num patches: 149049
Patch fields:
    id:           fiftyone.core.fields.ObjectIdField
    sample_id:    fiftyone.core.fields.ObjectIdField
    frame_id:     fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    frame_number: fiftyone.core.fields.FrameNumberField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detection)
    uniqueness:   fiftyone.core.fields.FloatField
View stages:
    1. ToFrames(config={'sample_frames': True})
    2. ToPatches(field='detections', config=None)

In [13]:
session = fo.launch_app(view)

In [14]:
for sample in view:
    if sample.detections.tags:
        print(sample)

In [15]:
view

Dataset:     2023.08.09.10.23.06
Media type:  image
Num patches: 149049
Patch fields:
    id:           fiftyone.core.fields.ObjectIdField
    sample_id:    fiftyone.core.fields.ObjectIdField
    frame_id:     fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    frame_number: fiftyone.core.fields.FrameNumberField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detection)
    uniqueness:   fiftyone.core.fields.FloatField
View stages:
    1. ToFrames(config={'sample_frames': True})
    2. ToPatches(field='detections', config=None)

In [16]:
data=patches_dataset.to_dict()

 100% |███████████| 149049/149049 [53.2s elapsed, 0s remaining, 3.0K samples/s]       


In [17]:
x=data['samples'][0]['detections']
type(x)
# set(x)
x

{'_id': {'$oid': '64d38f5e80960196cf7f2bf9'},
 '_cls': 'Detection',
 'attributes': {},
 'tags': [],
 'label': 'cellular_telephone',
 'bounding_box': [0.7294583,
  0.00023257956,
  0.08345804000000001,
  0.15950665044],
 'confidence': 0.93284637,
 'index': 0}

In [18]:
x=fo.Dataset.from_dict(data)

 100% |███████████| 149049/149049 [1.1m elapsed, 0s remaining, 2.0K samples/s]       


In [19]:
for sample in x:
    if sample.tags:
        print(sample.tags, end=' ')
    # if sample.detections.tags:
    #     print(sample.detections.tags, end=' ')

In [20]:
# type(x)
# len(x)

In [24]:
patches_dataset.export(
    export_dir=export_dataset,
    dataset_type=FiftyOneVideoLabelsDataset,
    label_field="detections",
    export_media="symlink",
)

Directory '/datasets/Milly/object_labels' already exists; export will be merged with existing files


AttributeError: 'NoneType' object has no attribute 'items'

In [25]:
ls /datasets/Milly/object_labels/labels -lah

total 37M
drwxrwxr-x 2 bea bea 4.0K Aug  4 09:07 ./
drwxrwxr-x 4 bea bea 4.0K Aug  4 09:07 ../
-rw-rw-r-- 1 bea bea 4.9M Aug  4 09:07 A_pin_mevo_0_video-0000_pv.json
-rw-rw-r-- 1 bea bea 1.2M Aug  4 09:07 A_pin_mevo_0_video-0001_pv.json
-rw-rw-r-- 1 bea bea 3.6M Aug  4 09:07 A_pin_mevo_0_video-0002_pv.json
-rw-rw-r-- 1 bea bea 3.7M Aug  4 09:07 A_pin_mevo_0_video-0003_pv.json
-rw-rw-r-- 1 bea bea 4.3M Aug  4 09:07 A_pin_mevo_0_video-0004_pv.json
-rw-rw-r-- 1 bea bea 781K Aug  4 09:07 A_pin_mevo_0_video-0005_pv.json
-rw-rw-r-- 1 bea bea 5.6M Aug  4 09:07 A_pin_mevo_0_video-0006_pv.json
-rw-rw-r-- 1 bea bea 3.9M Aug  4 09:07 A_pin_mevo_0_video-0007_pv.json
-rw-rw-r-- 1 bea bea 4.0M Aug  4 09:07 A_pin_mevo_0_video-0008_pv.json
-rw-rw-r-- 1 bea bea 4.8M Aug  4 09:07 A_pin_mevo_0_video-0009_pv.json
